In [1]:
import requests
import re 
import time
import json
from pymongo import MongoClient
import random

client = MongoClient('localhost', 27017)

# let's get the list of DBs already defined
print(client.list_database_names())

# access token can be used till february 2024
access_token = "EAAKb83xTK40BO7gBDVcd3GoFsLMwJFEv1MrkbdbqpTKOVeA2a0GuZCnusq2SuGeD4IG2GjbeEV5OkV4dzHEN53KTRoDgnsKlsPLAIkuo2Yz7zdtEYRl3IvM4mHct3e6syQu0MjKt4wq3FViWZBkms1jGhxbFmOe8ZALYRQNfOBTOs4wH9I5v4kC"
instagram_account_id = 17841402187545471


graph_url = 'https://graph.facebook.com/v18.0/'
def get_hashtag_id(hashtag = '',instagram_account_id = '',access_token = ''):
    url = graph_url + 'ig_hashtag_search'
    param = dict()
    param['user_id'] = instagram_account_id
    param['q'] = hashtag
    param['access_token'] = access_token
    response = requests.get(url,param)
    usage = json.loads(response.headers['x-app-usage'])
    print(usage)
    response = response.json()
    hashtag_id = response['data'][0]['id']
    return hashtag_id

# retrieve name and id for a hasthag
def get_data_of_hashtag(hashtag_id = '',access_token=''):
    url = graph_url + hashtag_id
    param =dict()
    param['access_token'] = access_token
    param['fields'] = 'id,name'
    response = requests.get(url,param)
    response = response.json()
    return response

# recent media for a hashtag
def get_recent_media(hashtag_id='',access_token = '',instagram_account_id = ''):
    url = graph_url + hashtag_id + '/recent_media'
    param = dict()
    param['access_token'] = access_token
    param['user_id'] = instagram_account_id
    param['fields'] = 'caption,like_count,permalink,timestamp,media_type,comments_count,media_url'
    response = requests.get(url, param,timeout=90)
    response = response.json()
    print(response)
    return response

def get_top_media(hashtag_id='',access_token='',instagram_account_id=''):
    url = graph_url + hashtag_id + '/top_media'
    param = dict()
    param['access_token'] = access_token
    param['user_id'] = instagram_account_id
    param['fields'] = 'caption,like_count,permalink'
    response = requests.get(url, param)
    response = response.json()
    print(response)
    return response

def get_all_posts_since_24hours(hashtag_names, access_token, instagram_account_id):
    posts = []
    hashtag_ids = []
    wait_for_decreaseing_limits()
    for name in hashtag_names:
        hashtag_id = get_hashtag_id(name,instagram_account_id,access_token)
        hashtag_ids.append(hashtag_id)
    
    random.shuffle(hashtag_ids)
    
    for hashtag_id in hashtag_ids:
        # print to verify the id
        #wait_for_decreaseing_limits()
        print("data for hashtag")
        print(get_data_of_hashtag(hashtag_id,access_token))
        posts_for_hashtag = get_all_posts_since_24hours_for_given_hashtag_id(hashtag_id,access_token, instagram_account_id)
        client['bda']['posts'].insert_many(posts_for_hashtag)
        posts = posts + posts_for_hashtag
    return posts


#def plot_most_common_hashtags_in_data()
def wait_for_decreaseing_limits():
    a = 1
    # try:
    #     param = dict()
    #     param['access_token'] = access_token
    #     response = requests.get(graph_url+"me/",param,timeout=300)
    #     # Here we look at three metircs total time, total cpu time and call count that give us percentages  for their limit usages.
    #     # the code get's delayed one hour, till they reset
    #     usage = json.loads(response.headers['x-app-usage'])
    #     print(usage)
    #     if usage["total_cputime"] >= 90 or usage["total_time"] >= 90 or usage["call_count"] >= 90:
    #         print("sleeping")
    #         time.sleep(600)
    # except requests.exceptions.Timeout:
    #     print("Timeout")

def get_all_posts_since_24hours_for_given_hashtag_id(hashtag_id,access_token,instagram_account_id):
    data = []
    count = 0
    #wait_for_decreaseing_limits()
    recent = get_recent_media(hashtag_id, access_token,instagram_account_id)
    data = [] + recent["data"]
    while True:
        if not ( 'paging' in recent and 'next' in recent['paging']):
            break
        count = count + 1
        try:
            response = requests.get(recent['paging']['next'] ,timeout=300)
            #if (count%50) == 0:
                #wait_for_decreaseing_limits()
            
            recent = response.json()
            # we saw that sometimes there is at the last pages not data returned, but urls for new pages
            if'data'not in recent:
                break
            print(recent['data'][0]['timestamp'])
            data = data + recent['data']
        except requests.exceptions.Timeout:
            print("Timeout")
            print(recent)
            break
        except: #code to run if error occurs
            print("caught_it")
            print(recent)
    return data
    
    


['admin', 'bda', 'config', 'local']


In [39]:
wait_for_decreaseing_limits()


{'call_count': 22, 'total_cputime': 0, 'total_time': 9}


In [5]:
from pymongo import MongoClient
import json

client = MongoClient('localhost', 27017)

# let's get the list of DBs already defined
print(client.list_database_names())

['admin', 'bda', 'config', 'local']


In [3]:
# let's create a new DB - bda
database = client['bda']

# let's create a collection
posts = database.posts

In [2]:

posts = get_all_posts_since_24hours(['war'],access_token,instagram_account_id)


{'call_count': 0, 'total_cputime': 0, 'total_time': 0}
data for hashtag
{'id': '17843857138016461', 'name': 'war'}
{'data': [{'caption': '#palestine #palestine🇵🇸 #gaza #gazaunderattack #freepalestine #savepalestine #falastin #فلسطين #غزة #occupation #israel #ceasefirenow🇵🇸 #🇵🇸 #🇵🇸فلسطين #🍉 hashtags for the algorithm: #cake #game #war #Facebook #smile #meme #funny #joke #smart #middleeast #aircraft #art #ball #football #basketball', 'like_count': 0, 'permalink': 'https://www.instagram.com/reel/C1rj4kqNhWC/', 'timestamp': '2024-01-04T13:50:47+0000', 'media_type': 'VIDEO', 'comments_count': 0, 'media_url': 'https://scontent-bcn1-1.cdninstagram.com/o1/v/t16/f1/m82/A34BE86D66CDF19F19445CC083E89D9A_video_dashinit.mp4?efg=eyJ2ZW5jb2RlX3RhZyI6InZ0c192b2RfdXJsZ2VuLmNsaXBzLnVua25vd24tQzMuNzIwLmRhc2hfYmFzZWxpbmVfMV92MSJ9&_nc_ht=scontent-bcn1-1.cdninstagram.com&_nc_cat=103&vs=1038895080671955_3697813079&_nc_vs=HBksFQIYT2lnX3hwdl9yZWVsc19wZXJtYW5lbnRfcHJvZC9BMzRCRTg2RDY2Q0RGMTlGMTk0NDVDQzA4M0U4OUQ5

I searched after the popularst hastags for the palätina isreal war. So I have searched 5 different hastags for the palästina side and 5 for isreal. For the palästina side I fount the hashtags #FreePalestine, #palistina, #savepalestine, #alquds (the holy city), gazaunderattack, #palestinewillbefree, #palastine. For the isreal side I have : #IsraelUnderAttack, #FreeIsrael, #israel, #proisrael (30000), #standwithisreal, #BringThemHomeNow. I will put all posts in one collection into a mongodb database.Also we take the hashtag #war to see a more broader overview of what is discussed under this hashtag.

In [4]:
# get data for hashtags
# hashtags_for_palästina = ['freepalestine', 'palestine', 'savepalestine','palestinewillbefree','gazaunderattack','gaza']
# hashtags_for_isreal = ['IsraelUnderAttack', 'prayforisrael', 'israel', 'proisrael', 'standwithisrael', 'BringThemHomeNow']
# hashtags_for_general_war = ['war']

hashtags = ["gaza","israel","palestine","war","freepalestine","BringThemHomeNow","standwithisrael"]




In [4]:
while True:
    posts = get_all_posts_since_24hours(hashtags_for_palästina+hashtags_for_isreal+hashtags_for_general_war,access_token, instagram_account_id)
    print(f"posts succesfull getted {len(posts)}")
    print(f"posts succesfull getted {client['bda']['posts'].count_documents({})}")
    #sleep 3 hours till next execution
    time.sleep(10800)


{'call_count': 20, 'total_cputime': 0, 'total_time': 41}
{'call_count': 25, 'total_cputime': 0, 'total_time': 43}
data for hashtag
{'id': '17841596584074200', 'name': 'israel'}
{'call_count': 27, 'total_cputime': 0, 'total_time': 43}
{'data': [{'caption': 'Für Amine war der Gaza-Streifen immer weit weg - sie wuchs als Tochter eines Palästinensers in Deutschland auf. Doch seit dem Terroranschlag der radikal-islamischen Hamas auf Israel wird sie immer wieder nach ihrer Meinung zum Nahostkonflikt gefragt und soll Stellung beziehen, obwohl sie nicht mehr Informationen habe als die anderen, sagt sie. Und dennoch: Aufgrund ihres palästinensischen Hintergrundes empfindet sie es als ihre Pflicht, zu vermitteln. Amine möchte Lehrerin werden und Kinder im Kampf gegen Antisemitismus aufklären. \n \nIn den Medien und der öffentlichen Diskussion vermisst sie oft die palästinensische Seite. "Das ist ungerecht und vermittelt Menschen mit palästinensischen Wurzeln das Gefühl, ausgegrenzt und benachtei

ConnectionError: HTTPSConnectionPool(host='graph.facebook.com', port=443): Max retries exceeded with url: /v18.0/me/?access_token=EAAKb83xTK40BO7gBDVcd3GoFsLMwJFEv1MrkbdbqpTKOVeA2a0GuZCnusq2SuGeD4IG2GjbeEV5OkV4dzHEN53KTRoDgnsKlsPLAIkuo2Yz7zdtEYRl3IvM4mHct3e6syQu0MjKt4wq3FViWZBkms1jGhxbFmOe8ZALYRQNfOBTOs4wH9I5v4kC (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x00000142266E7210>: Failed to resolve 'graph.facebook.com' ([Errno 11001] getaddrinfo failed)"))

Due to the execution fails and we are not able to get the data for all hashtags. We will now reduce the amlunt of hashtags and also put more manual work in it. Therefore we exported the data of the collection, droped the collection with the tool mongo db compass. And now we create it again

In [12]:
# let's create a new DB - bda
database = client['bda']

# let's create a collection
posts = database.posts



In [13]:
# I think we first have to insert some documents in the collection
print(client.list_database_names())

['admin', 'config', 'local']


In [9]:
time.sleep(3600)
posts = get_all_posts_since_24hours([hashtags[0]],access_token, instagram_account_id)

{'call_count': 0, 'total_cputime': 0, 'total_time': 1}
data for hashtag
{'id': '17843784049044435', 'name': 'gaza'}
{'data': [{'caption': "1.1.2023 Gaza City under attack' via eye.on.palestine #gaza #gazaunderattack #gazagenocide #savegaza #stopisraieliangenocide #savechildrenpalestine #savepalestine #palestine🇵🇸 #islam #instagram", 'like_count': 0, 'permalink': 'https://www.instagram.com/reel/C1lRZjpy6mK/', 'timestamp': '2024-01-02T03:15:43+0000', 'media_type': 'VIDEO', 'comments_count': 0, 'media_url': 'https://scontent-mad2-1.cdninstagram.com/o1/v/t16/f1/m82/9C4140638E0D4A47EB1F6C053D71148A_video_dashinit.mp4?efg=eyJ2ZW5jb2RlX3RhZyI6InZ0c192b2RfdXJsZ2VuLmNsaXBzLnVua25vd24tQzMuNzIwLmRhc2hfYmFzZWxpbmVfMV92MSJ9&_nc_ht=scontent-mad2-1.cdninstagram.com&_nc_cat=102&vs=745962597393259_192487747&_nc_vs=HBksFQIYT2lnX3hwdl9yZWVsc19wZXJtYW5lbnRfcHJvZC85QzQxNDA2MzhFMEQ0QTQ3RUIxRjZDMDUzRDcxMTQ4QV92aWRlb19kYXNoaW5pdC5tcDQVAALIAQAVAhg6cGFzc3Rocm91Z2hfZXZlcnN0b3JlL0dDWER3eGhETXE4a1dGd0NBTTg5M1pZQWR

In [10]:
time.sleep(3600)
posts = get_all_posts_since_24hours([hashtags[1]],access_token, instagram_account_id)

{'call_count': 0, 'total_cputime': 0, 'total_time': 1}
data for hashtag
{'id': '17841596584074200', 'name': 'israel'}
{'data': [{'caption': "Rocket Attack on New Year's Eve !! Not even a minute after midnight struck here in Israel, on New Year's Eve, H*amas attacked with massive rocket fire. As I was at a gathering where people tried to celebrate like the rest of the world, to try and break away from the intensity and trauma of war...H*amas reminded us that there is no escape. And, there will be no escape, until we completely eradicate their t*errorism.\n\n#Israel #IsraelAtWar #Hamas #WarOnTear #NewYearsEve #BreakingNews #IsraelNews #RocketFire #IronDome #IADN", 'like_count': 0, 'permalink': 'https://www.instagram.com/reel/C1lbbQVS9oR/', 'timestamp': '2024-01-02T04:48:09+0000', 'media_type': 'VIDEO', 'comments_count': 0, 'media_url': 'https://scontent-mad2-1.cdninstagram.com/o1/v/t16/f1/m82/D248B9C2646380CFCF3A777D8305DD82_video_dashinit.mp4?efg=eyJ2ZW5jb2RlX3RhZyI6InZ0c192b2RfdXJsZ2Vu

In [11]:
time.sleep(3600)
posts = get_all_posts_since_24hours([hashtags[2]],access_token, instagram_account_id)

{'call_count': 0, 'total_cputime': 0, 'total_time': 1}
data for hashtag
{'id': '17843870389043902', 'name': 'palestine'}
{'data': [{'caption': 'FREE PALESTINA 🇸🇩 #palestine #palestine🇵🇸 #palestina #PALESTINA', 'like_count': 0, 'permalink': 'https://www.instagram.com/p/C1llPKSLdTi/', 'timestamp': '2024-01-02T06:06:38+0000', 'media_type': 'IMAGE', 'comments_count': 0, 'media_url': 'https://scontent-mad1-1.cdninstagram.com/v/t51.29350-15/416280576_1412788522929633_1135091189462273380_n.webp?stp=dst-jpg&_nc_cat=107&ccb=1-7&_nc_sid=c4dd86&_nc_ohc=eI25K3GYBuwAX_3GF_Z&_nc_ht=scontent-mad1-1.cdninstagram.com&edm=AEoDcc0EAAAA&oh=00_AfCJsXOcN87Pbu-XspkXqKOMgWHOPS6MYc2_OQwH00bcJw&oe=65997F83', 'id': '17977829675634631'}, {'caption': '#aksatufanı #gazzedesoykırımvar\n#boykot #gazzedekatliamvar #palestine #filistin🇵🇸 #kudus\n#mescidiaksa \n.\n.\n.\n.\n.\n.\n.\n.\n.\n.\n.\n.\n.\n.\n.\n.\n\n.\n\n#duâ #dualar #dua  #hzmuhammed #peygamber #namaz  #ayet #secde #ayetikerime #ayetler #hadis #hadisişerif #

In [8]:
time.sleep(3600)
posts = get_all_posts_since_24hours([hashtags[3]],access_token, instagram_account_id)

{'call_count': 0, 'total_cputime': 0, 'total_time': 0}
data for hashtag
{'id': '17843857138016461', 'name': 'war'}
{'data': [{'caption': '#royal #london #artist #singer #music #inspiration #motivation #power #God #love #pray #war #muslim #islam #jews #isreal #gaza #palestine #turkey #europe #dubai #saudiarabia #iran #peace #jesus #christian #prophetmuhammadﷺ  #war #pray', 'like_count': 0, 'permalink': 'https://www.instagram.com/p/C1lJsmyCw8y/', 'timestamp': '2024-01-02T02:06:00+0000', 'media_type': 'IMAGE', 'comments_count': 0, 'media_url': 'https://scontent-mad2-1.cdninstagram.com/v/t51.29350-15/416532550_749036803256887_7955065434581007307_n.jpg?_nc_cat=111&ccb=1-7&_nc_sid=c4dd86&_nc_ohc=1ZoC_XF7MSMAX9EFAAM&_nc_ht=scontent-mad2-1.cdninstagram.com&edm=AEoDcc0EAAAA&oh=00_AfDJ9xrkYCSK8lCZz5RMDxnHg3JULd8rI0xBmFBc-YQqTQ&oe=65990FB8', 'id': '17992151822294365'}, {'caption': '#freeprisonersofwar #freeazovstaldefenders\n#freeazov#ave_eam #russia_delenda_est #war #ukraine #war_in_ukraine #naz

In [5]:
time.sleep(3600)
posts = get_all_posts_since_24hours([hashtags[4]],access_token, instagram_account_id)

{'call_count': 0, 'total_cputime': 0, 'total_time': 0}
data for hashtag
{'id': '17841562354125930', 'name': 'freepalestine'}
{'data': [{'caption': 'Immaginate esultare perché funziona l’elettricità.\nTerror¡mo non sono solo le b0mbe, ma privare di elettricità, cibo, acqua e gas un’intera popolazione, aprendo e chiudendo i rubinetti come fosse un gioco. \n#buonanno #2024 #freepalestine #gaza #isisrael #israeleterrorista', 'like_count': 0, 'permalink': 'https://www.instagram.com/reel/C1kuSmsodY9/', 'timestamp': '2024-01-01T22:08:08+0000', 'media_type': 'VIDEO', 'comments_count': 0, 'media_url': 'https://scontent-mad2-1.cdninstagram.com/o1/v/t16/f1/m82/3F4F2B35E89DEBE82AC06E3C964BCE8E_video_dashinit.mp4?efg=eyJ2ZW5jb2RlX3RhZyI6InZ0c192b2RfdXJsZ2VuLmNsaXBzLnVua25vd24tQzMuNzIwLmRhc2hfYmFzZWxpbmVfMV92MSJ9&_nc_ht=scontent-mad2-1.cdninstagram.com&_nc_cat=101&vs=1567330860760007_148885458&_nc_vs=HBksFQIYT2lnX3hwdl9yZWVsc19wZXJtYW5lbnRfcHJvZC8zRjRGMkIzNUU4OURFQkU4MkFDMDZFM0M5NjRCQ0U4RV92aWRlb19k

In [6]:
time.sleep(3600)
posts = get_all_posts_since_24hours([hashtags[5]],access_token, instagram_account_id)

{'call_count': 0, 'total_cputime': 0, 'total_time': 0}
data for hashtag
{'id': '17841531883073197', 'name': 'bringthemhomenow'}
{'data': [{'caption': 'Mia Schem, 21-years-old, is finally able to speak truthfully about her time in captivity without Hamas cameras pointed at her. \n\n#israel #palestine #gaza #freegaza #freegazafromhamas #freepalestine #terrorism #standwithisrael #istandwithisrael #standwithpalestine #humansofjudaism #humansofpalestine #fromtherivertothesea #occupation #jihad #bringthemhome #bringthemhomenow #freethehostages #humans #humanitarian #crisis #humanitariancrisis #releasethehostages #peace #prayforpeace #prayforisrael #prayforpalestine #prayforgaza', 'like_count': 0, 'permalink': 'https://www.instagram.com/reel/C1k7Wj-PCZ4/', 'timestamp': '2024-01-02T00:00:49+0000', 'media_type': 'VIDEO', 'comments_count': 0, 'media_url': 'https://scontent-mad2-1.cdninstagram.com/o1/v/t16/f1/m82/7346832B7268516020AD7AE1E8C652AA_video_dashinit.mp4?efg=eyJ2ZW5jb2RlX3RhZyI6InZ0c192

In [7]:
time.sleep(3600)
posts = get_all_posts_since_24hours([hashtags[6]],access_token, instagram_account_id)

{'call_count': 0, 'total_cputime': 0, 'total_time': 0}
data for hashtag
{'id': '17841562501088775', 'name': 'standwithisrael'}
{'data': [{'caption': "They are too much 🤬🇮🇱💩\nSpread it share it and don't stop talking about PALESTINE 🇵🇸\n#freepalestine\n#stopgenocide\n#genocideingaza\n#gazaunderattack\n#gaza\n#ceasefire#now\n🇵🇸 🇵🇸 🇵🇸 🇵🇸 🇵🇸 🇵🇸 🇵🇸 🇵🇸 🇵🇸 🇵🇸 🇵🇸 🇵🇸 🇵🇸 🇵🇸 🇵🇸 🇵🇸 🇵🇸 🇵🇸\n\nخيبر خيبر يا يهود جيش محمد سوف يعود.\nKhaybar Khaybar oh Jews, the army of Muhammad will return\nAl-Aqsa is a Capital of Palestine.\n#Israelcuak\n#Israelkoyak\n#TaufanAlAqsa\nI don't #loveisrael\nNever #loveisraelforever\nHate #israelforever\nNever #standwithisrael\nNever #supportisrael\nDon't #visitisrael\n🇮🇱 💩 🇮🇱 💩 🇮🇱 💩 🇮🇱 💩 🇮🇱 💩 🇮🇱", 'like_count': 0, 'permalink': 'https://www.instagram.com/reel/C1lCP-YP4Sx/', 'timestamp': '2024-01-02T01:03:01+0000', 'media_type': 'VIDEO', 'comments_count': 0, 'media_url': 'https://scontent-mad2-1.cdninstagram.com/o1/v/t16/f1/m82/FA44D489B288E4ADA72FA13B3AAD0A88_video_dashini

In [12]:
client['bda']['posts'].count_documents({})

709031

In [12]:
client['bda']['posts'].insert_many(posts)

In [13]:
client['bda']['posts'].

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'bda'), 'posts')

In [12]:
client['bda']['posts'].count_documents({})

71012